做一個adr套利表

美股adr列表
台股股價
匯率
換算機制

In [1]:

import requests
import pandas as pd

def toPd(url,parameter):
    data=requests.get(url,params=parameter)
    data=data.json()
    data=pd.DataFrame(data['data'])
    return data



In [5]:
import datetime

def beforeDays(days):
    today=datetime.date.today()
    beforeDays=datetime.timedelta(days=days)
    return today-beforeDays

In [6]:
# import Common
# import Datetime
import pandas as pd

# before days
beforeDays=beforeDays(2)

# exchange rate pd
exchangeurl="https://api.finmindtrade.com/api/v4/data"
exchangeParam={
    "dataset": "TaiwanExchangeRate",
    "data_id": "USD",
    "start_date":beforeDays
}
exchange=toPd(exchangeurl,exchangeParam)

#adrlist
adrMap={"TSM":"2330","UMC":"2303","CHT":"2412","ASX":"3711","IMOS":"8150"}

#adr multiple list
changeMul={
    "stock_id":["2330","2303","2412","3711","8150"],
    "換股數":[5,5,10,2,20]      
          }
changeMul=pd.DataFrame(changeMul)

In [7]:
# for item in adrList concat item

adrInfo=[]

for item in adrMap:
    adrurl="https://api.finmindtrade.com/api/v3/data"
    adrParam={
        "dataset": "USStockPrice",
        "stock_id": item,
        "date": beforeDays,
    }
    adrItem=toPd(adrurl,adrParam)
    adrInfo.append(adrItem)

adrInfo=pd.concat(adrInfo)
print(adrInfo)

   Adj_Close  Close   High    Low   Open   Volume        date stock_id
0      91.50  91.50  91.91  90.65  91.02  6256259  2022-05-23      TSM
0       8.55   8.55   8.57   8.44   8.50  4802874  2022-05-23      UMC
0      42.13  42.13  42.25  41.89  41.89    84890  2022-05-23      CHT
0       6.70   6.70   6.72   6.58   6.65  5715143  2022-05-23      ASX
0      32.61  32.61  32.67  32.29  32.29     7752  2022-05-23     IMOS


In [8]:
adrInfo['stock_id']=adrInfo['stock_id'].map(adrMap)

print(adrInfo)

   Adj_Close  Close   High    Low   Open   Volume        date stock_id
0      91.50  91.50  91.91  90.65  91.02  6256259  2022-05-23     2330
0       8.55   8.55   8.57   8.44   8.50  4802874  2022-05-23     2303
0      42.13  42.13  42.25  41.89  41.89    84890  2022-05-23     2412
0       6.70   6.70   6.72   6.58   6.65  5715143  2022-05-23     3711
0      32.61  32.61  32.67  32.29  32.29     7752  2022-05-23     8150


In [9]:
twstockInfo=[]

for item in adrMap:
    twstockurl="https://api.finmindtrade.com/api/v4/data"
    twstockParam={
        "dataset": "TaiwanStockPrice",
        "data_id": adrMap[item],
        "start_date": beforeDays,
        "token": "", # 參考登入，獲取金鑰
    }
    twstock=toPd(twstockurl,twstockParam)
    twstockInfo.append(twstock)

twstockInfo=pd.concat(twstockInfo)
print(twstockInfo)

         date stock_id  Trading_Volume  Trading_money    open    max     min  \
0  2022-05-23     2330        18919447    10040778291  533.00  534.0  528.00   
0  2022-05-23     2303        51326592     2598202133   51.20   51.4   50.10   
0  2022-05-23     2412         8794677     1092165911  125.00  125.0  123.50   
0  2022-05-23     3711         6493889      633868175   98.50   98.7   97.00   
0  2022-05-23     8150         1940547       93448463   48.55   48.6   47.95   

   close  spread  Trading_turnover  
0  528.0   -2.00             18775  
0   50.3   -0.40             22417  
0  123.5   -1.00              3510  
0   97.5    0.00              4193  
0   48.0   -0.35              1819  


In [10]:
#組合data
adrCompare=pd.merge(twstockInfo,adrInfo,how="outer")
adrCompare=pd.merge(adrCompare,exchange,how="outer")
adrCompare=pd.merge(adrCompare,changeMul,how="outer")

print(adrCompare)

         date stock_id  Trading_Volume  Trading_money    open    max     min  \
0  2022-05-23     2330        18919447    10040778291  533.00  534.0  528.00   
1  2022-05-23     2303        51326592     2598202133   51.20   51.4   50.10   
2  2022-05-23     2412         8794677     1092165911  125.00  125.0  123.50   
3  2022-05-23     3711         6493889      633868175   98.50   98.7   97.00   
4  2022-05-23     8150         1940547       93448463   48.55   48.6   47.95   

   close  spread  Trading_turnover  ...   High    Low   Open   Volume  \
0  528.0   -2.00             18775  ...  91.91  90.65  91.02  6256259   
1   50.3   -0.40             22417  ...   8.57   8.44   8.50  4802874   
2  123.5   -1.00              3510  ...  42.25  41.89  41.89    84890   
3   97.5    0.00              4193  ...   6.72   6.58   6.65  5715143   
4   48.0   -0.35              1819  ...  32.67  32.29  32.29     7752   

   currency  cash_buy cash_sell  spot_buy  spot_sell  換股數  
0       USD     29.2

In [11]:
adrCompare['ADR換算']=adrCompare.Close/adrCompare.換股數*(adrCompare.spot_buy+adrCompare.spot_sell)/2
adrCompare['ADR溢價']=((adrCompare.ADR換算-adrCompare.close)/adrCompare.close).apply(lambda x:format(x,'.2%'))
list=['date',
     'stock_id',
     'close',
     'Close',
     '換股數',
     'ADR換算',
     'ADR溢價']

adrCompare.sort_values(by=["date"])
pd.set_option('display.max_rows',None)
print(adrCompare[list].sort_values(by=["stock_id","date"]))

         date stock_id  close  Close  換股數      ADR換算  ADR溢價
1  2022-05-23     2303   50.3   8.55    5   50.65020  0.70%
0  2022-05-23     2330  528.0  91.50    5  542.04600  2.66%
2  2022-05-23     2412  123.5  42.13   10  124.78906  1.04%
3  2022-05-23     3711   97.5   6.70    2   99.22700  1.77%
4  2022-05-23     8150   48.0  32.61   20   48.29541  0.62%


In [36]:
list(adrCompare)

['date',
 'stock_id',
 'Trading_Volume',
 'Trading_money',
 'open',
 'max',
 'min',
 'close',
 'spread',
 'Trading_turnover',
 'Adj_Close',
 'Close',
 'High',
 'Low',
 'Open',
 'Volume',
 'currency',
 'cash_buy',
 'cash_sell',
 'spot_buy',
 'spot_sell']